In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Simulation parameters (constants that define the system)
L = 20.0  # Length of the simulation box (assumed cubic, so it's LxLxL).
N = 40  # Number of particles in the simulation.
T = 1.0  # Temperature of the system (used to initialize velocities).
sigma = 1.0  # Lennard-Jones parameter for distance scale.
epsilon = 1.0  # Lennard-Jones parameter for energy scale (depth of the potential well).
mass = 1.0  # Mass of each particle (assumed uniform for simplicity).
rcut = 2.5 * sigma  # Cutoff distance for the Lennard-Jones potential.
dt = 0.005  # Time step for integration (small steps ensure accuracy).
steps = 1000  # Number of simulation steps to run.

ModuleNotFoundError: No module named 'numpy'

In [3]:
# --- Initialization Functions ---

def initialize_random(L, N):
    """Initializes particles with random positions within the box."""
    return np.random.uniform(-L / 2, L / 2, (N, 3))

def initialize_fcc(L, N):
    """Initializes particles on an FCC lattice."""
    n_cells = int(np.ceil((N / 4)**(1/3)))
    a = L / n_cells
    base_positions = np.array([
        [0, 0, 0],
        [0, a/2, a/2],
        [a/2, 0, a/2],
        [a/2, a/2, 0]
    ])
    positions = []
    for i in range(n_cells):
        for j in range(n_cells):
            for k in range(n_cells):
                cell_origin = np.array([i*a, j*a, k*a])
                for pos in base_positions:
                    positions.append(cell_origin + pos)
    positions = np.array(positions[:N])
    positions -= L / 2
    return positions

def initialize_gaussian(L, N, sigma):
    """Initializes particles with a Gaussian distribution."""
    positions = np.random.normal(0, sigma, (N, 3))
    positions = apply_pbc(positions)  # Apply PBCs
    return positions

def initialize_from_file(filename):
    """Loads particle positions from a file."""
    positions = np.loadtxt(filename)
    return positions

# --- Choose an Initialization Method ---
# Uncomment the line corresponding to your desired initialization method

# positions = initialize_random(L, N)
# positions = initialize_fcc(L, N) 
positions = initialize_gaussian(L, N, sigma=2.0)
# positions = initialize_from_file('initial_positions.txt')

NameError: name 'L' is not defined

In [4]:
# --- Simulation Functions ---

# Initialize particle velocities using the Maxwell-Boltzmann distribution
# This gives velocities corresponding to the specified temperature
velocities = np.random.normal(0, np.sqrt(T / mass), (N, 3))  # Gaussian distribution for velocities.

# Placeholder for forces; initially set to zero for all particles
forces = np.zeros_like(positions)

# Function to apply periodic boundary conditions
def apply_pbc(positions):
    """
    Ensures particles remain inside the simulation box by wrapping their positions
    if they leave the boundaries. Uses modular arithmetic to "teleport" particles
    back into the box.
    """
    return positions - L * np.round(positions / L)

# Function to calculate pairwise distances using the minimum image convention
def compute_distances(positions):
    """
    Computes the distance between every pair of particles while considering
    periodic boundaries. Returns a 3D array of displacement vectors.
    """
    distances = np.zeros((N, N, 3))  # Array to store pairwise displacement vectors.
    for i in range(N):
        for j in range(i + 1, N):  # Avoid redundant calculations (j > i ensures unique pairs).
            delta = positions[i] - positions[j]  # Displacement vector between particles i and j.
            delta -= L * np.round(delta / L)  # Adjust using periodic boundary conditions.
            distances[i, j] = delta
            distances[j, i] = -delta  # Symmetric matrix: if distance[i,j] = +d, then distance[j,i] = -d.
    return distances

# Function to calculate forces and potential energy
def compute_forces_and_energy(positions):
    """
    Calculates the Lennard-Jones forces acting on each particle and the total potential energy.
    """
    forces.fill(0.0)  # Reset forces to zero before recalculating.
    potential_energy = 0.0  # Initialize potential energy to zero.
    distances = compute_distances(positions)  # Get displacement vectors for all pairs.

    for i in range(N):
        for j in range(i + 1, N):
            r_vec = distances[i, j]  # Displacement vector between particles i and j.
            r = np.linalg.norm(r_vec)  # Magnitude of the distance (Euclidean norm).
            if r < rcut:  # Only consider pairs within the cutoff distance.
                r_inv = sigma / r  # Inverse distance.
                r_inv6 = r_inv**6  # (sigma / r)^6
                r_inv12 = r_inv6**2  # (sigma / r)^12
                potential_energy += 4 * epsilon * (r_inv12 - r_inv6)  # Lennard-Jones potential.

                # Compute Lennard-Jones force magnitude
                force_magnitude = 24 * epsilon * (2 * r_inv12 - r_inv6) / r
                forces[i] += force_magnitude * r_vec  # Add force vector to particle i.
                forces[j] -= force_magnitude * r_vec  # Newton's third law: equal and opposite force on j.
    return forces, potential_energy

# Function to update positions and velocities using the Velocity Verlet algorithm
def velocity_verlet(positions, velocities, forces):
    """
    Performs one step of the Velocity Verlet algorithm:
    1. Update positions based on current velocities and forces.
    2. Calculate new forces based on updated positions.
    3. Update velocities based on average of old and new forces.
    """
    # Step 1: Update positions
    positions += velocities * dt + 0.5 * forces * dt**2
    positions = apply_pbc(positions)  # Ensure particles stay within the box.

    # Step 2: Calculate new forces
    new_forces, potential_energy = compute_forces_and_energy(positions)

    # Step 3: Update velocities
    velocities += 0.5 * (forces + new_forces) * dt  # Average of old and new forces.

    return positions, velocities, new_forces, potential_energy

NameError: name 'np' is not defined

In [ ]:
# Main simulation loop
positions = apply_pbc(positions)  # Ensure initial positions are within the box.
forces, potential_energy = compute_forces_and_energy(positions)  # Calculate initial forces and energy.
kinetic_energies = []  # List to store kinetic energy at each time step.
potential_energies = []  # List to store potential energy at each time step.

for step in range(steps):
    # Update positions, velocities, and forces using Velocity Verlet
    positions, velocities, forces, potential_energy = velocity_verlet(positions, velocities, forces)

    # Calculate kinetic energy: (1/2)mv^2 for all particles
    kinetic_energy = 0.5 * mass * np.sum(velocities**2)
    kinetic_energies.append(kinetic_energy)
    potential_energies.append(potential_energy)

    # Output energy information every 100 steps
    if step % 100 == 0:
        print(f"Step {step}: Total Energy = {kinetic_energy + potential_energy:.3f}")

# Plot energy results
import matplotlib.pyplot as plt

plt.figure()
plt.plot(kinetic_energies, label='Kinetic Energy')
plt.plot(potential_energies, label='Potential Energy')
plt.plot(np.array(kinetic_energies) + np.array(potential_energies), label='Total Energy')
plt.xlabel('Time Step')
plt.ylabel('Energy')
plt.legend()
plt.title('Energy Conservation in Lennard-Jones Fluid')
plt.show()

In [ ]:
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import Image

# Function to animate particles and save as a GIF
def animate_box(positions_over_time, L):
    """
    Creates an animation of particles in the simulation box and saves it as a GIF.

    Parameters:
    - positions_over_time: List of particle positions at each timestep.
    - L: Simulation box size (assumed cubic, so it's LxLxL).
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Scatter plot for particle positions
    scatter = ax.scatter([], [], [], c='r', marker='o', s=10)

    # Draw the simulation box edges
    edges = [
        [[-L/2, -L/2, -L/2], [-L/2, -L/2, L/2]],
        [[-L/2, -L/2, -L/2], [-L/2, L/2, -L/2]],
        [[-L/2, -L/2, -L/2], [L/2, -L/2, -L/2]],
        [[-L/2, -L/2, L/2], [-L/2, L/2, L/2]],
        [[-L/2, -L/2, L/2], [L/2, -L/2, L/2]],
        [[-L/2, L/2, -L/2], [-L/2, L/2, L/2]],
        [[-L/2, L/2, -L/2], [L/2, L/2, -L/2]],
        [[-L/2, L/2, L/2], [L/2, L/2, L/2]],
        [[L/2, -L/2, -L/2], [L/2, -L/2, L/2]],
        [[L/2, -L/2, -L/2], [L/2, L/2, -L/2]],
        [[L/2, -L/2, L/2], [L/2, L/2, L/2]],
        [[L/2, L/2, -L/2], [L/2, L/2, L/2]],
    ]
    for edge in edges:
        ax.plot(*zip(*edge), c='k')  # Draw each edge of the box.

    # Set axis limits and labels
    ax.set_xlim(-L / 2, L / 2)
    ax.set_ylim(-L / 2, L / 2)
    ax.set_zlim(-L / 2, L / 2)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('3D Particle Simulation')

    # Update function for the animation
    def update(frame):
        scatter._offsets3d = (
            positions_over_time[frame][:, 0],
            positions_over_time[frame][:, 1],
            positions_over_time[frame][:, 2]
        )
        return scatter,

    # Create the animation
    ani = FuncAnimation(fig, update, frames=len(positions_over_time), interval=50, blit=False)

    # Save the animation as a GIF
    ani.save("simulation.gif", writer=PillowWriter(fps=20))

    print("Animation saved as 'simulation.gif'")
    return ani


# Example: Simulated positions over time
positions_over_time = []  # Initialize list to store positions at each step
positions = apply_pbc(positions)  # Ensure initial positions are inside the box
forces, potential_energy = compute_forces_and_energy(positions)  # Initial forces

for step in range(steps):
    positions, velocities, forces, potential_energy = velocity_verlet(positions, velocities, forces)
    positions_over_time.append(positions.copy())  # Store positions for animation

# Call the function to save the animation
ani = animate_box(positions_over_time, L)

# Display the animation as a GIF
Image("simulation.gif")